[View in Colaboratory](https://colab.research.google.com/github/rajagopalmotivate/DeepLearnIITMA1/blob/master/1B_Create_Model_and_save_it,_later_we_will_visualize_it_filters.ipynb)

In [0]:
!pip install pydot

In [0]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [0]:
import tensorflow as tf
from keras import Input
from keras import layers
from keras import datasets
import numpy as np
from keras import utils
from keras.models import  Model
from keras.utils import plot_model
import keras
import os
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
from sklearn.metrics import classification_report
from decimal import Decimal
import  matplotlib.pyplot as plt
from scipy.misc import toimage
from keras.preprocessing.image import ImageDataGenerator
from google.colab import files


In [0]:
#Input parameters are depthofnetwork, generalization capacity , 

def createANetwork(ExpNo, noofCNNlayers, noofDenselayers, noofCNNFilters, noofDensenodes  ,
                  networkwidthTamperRate, stride, maxpool, noofPARALLELbranches , kernalsize, dropoutrateinCNN=0, dropoutrateinFC=0.2):
    myinputShape = (32, 32, 3)
    myinputTensor = Input(shape=myinputShape, name='1')
    x = myinputTensor
    layerno = 1
    isskipaddingMaxPool = True
    
    for i in range(noofCNNlayers):
        x = layers.Conv2D(noofCNNFilters, kernel_size=(kernalsize,kernalsize), strides=(stride,stride), activation='relu', name='CNNlayer'+str(layerno)) (x)
        print( str(layerno) + ' Conv2D  Filters=' + str(noofCNNFilters) + '.    kernel size=(' + str(kernalsize) + ' , ' + str(kernalsize) + ' )'  + ' . stride=' + str(stride) + '     Relu')
        x = layers.BatchNormalization()(x)
        layerno = layerno +1
        if(layerno > 3):
            maxpool = 1
        if(layerno > 1):
            stride = 1
        if (isskipaddingMaxPool == False):
            x = layers.MaxPooling2D(pool_size=(maxpool, maxpool), name='MaxPool'+str(layerno)) (x)
            print( str(layerno) + ' MaxPool  . Poolsize=' + str(maxpool) )
            layerno = layerno +1
            x = layers.Dropout(dropoutrateinCNN) (x)
            print( '  Dropout '  + str(dropoutrateinCNN) )
            isskipaddingMaxPool = True
            noofCNNFilters = round( noofCNNFilters * networkwidthTamperRate )
            if(noofCNNFilters>512):
                noofCNNFilters = 512
        else:
            isskipaddingMaxPool = False


    x = layers.Flatten(name='Flatten' + str(layerno) ) (x)
    print(str(layerno) + ' Flatten  ' )

    layerno = layerno + 1

    for i in range(noofDenselayers):
        x = layers.Dense(noofDensenodes, activation='relu', name='Dense'+ str(layerno) )(x)
        print( str(layerno) + ' Dense  ' + str(noofDensenodes) )
        layerno = layerno +1
        x = layers.Dropout(dropoutrateinFC) (x)
        print( '  Dropout '  + str(dropoutrateinFC) )
        layerno = layerno +1
        noofDensenodes = round(noofDensenodes/2)

    myoutputTensor = layers.Dense(10, activation='softmax', name='DenseMultiClassSoftmaxLayer') (x)

    mymodel = Model(myinputTensor, myoutputTensor)

    print(mymodel.summary())

   # plot_model(mymodel, show_shapes=True, to_file='models'+str(ExpNo) + '.png')
   # plot_model(mymodel, show_shapes=False, to_file='m'+str(ExpNo) + '.png')


    return mymodel


In [0]:
def create_callbacks(expno):
   # tensorboard_callback = TensorBoard(log_dir=os.path.join(os.getcwd(), "log", "am18d301_Exp1" + str(expno) ), histogram_freq=1, batch_size=32,   write_graph=False, write_grads=False)
   # checkpoint_callback = ModelCheckpoint(filepath="./model-weights.{epoch:02d}.hdf5", monitor='val_acc', verbose=0, save_best_only=True)
    earlystop = EarlyStopping(monitor='val_acc', min_delta=0.0001, patience=1, verbose=1, mode='auto')
    return [ earlystop]

In [0]:
def print_model_metrics(model, xtest, ytest ):
    loss, accuracy = model.evaluate(x=xtest, y=ytest)
    print("\n model test loss is "+str(loss)+" accuracy is "+str(accuracy))

    y_softmax = model.predict(xtest)  # this is an n x class matrix of probabilities
    y_hat = y_softmax.argmax(axis=-1)  # this will be the class number.
    test_y = ytest.argmax(axis=-1)  # our test data is also categorical
    print(classification_report(test_y, y_hat))
    return [loss, accuracy]

In [0]:
def plot_imgs(X):
    plt.figure(1)
    k = 0
    for i in range(0,4):
        for j in range(0,4):
            plt.subplot2grid((4,4),(i,j))
            plt.imshow(toimage(X[k]))
            k = k+1
    # show the plot
    plt.show()

In [0]:
def convertprecision(alongfloat):
    alongfloat1 = Decimal(alongfloat * 100)
    alongfloat2 = round(alongfloat1, 2)
    myformatedfloat = float(alongfloat2)
    return myformatedfloat 

In [0]:
def runaexpeirment(ExpName, anetworkmodel):

    mymodel = anetworkmodel

    (xtrain1, ytrain1), (xtest, ytest) = datasets.cifar10.load_data()
    
    plot_imgs(xtrain1[:16])

    xtrain  = xtrain1[:42000, :]
    ytrain = ytrain1[:42000, :]

    xval = xtrain1[42000:, :]
    yval = ytrain1[42000:, :]

    datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
    datagen.fit(xtrain)
    

    print( 'Shape of CIFAR Training Set Inputs: ' + str(xtrain.shape) )
    print( 'Shape of CIFAR Training Set Labels: ' + str(ytrain.shape) )
    print( 'Shape of CIFAR Test Set Inputs: ' + str(xtest.shape) )
    print( 'Shape of CIFAR Validation Set Inputs: ' + str(xval.shape) )


    xtrain = xtrain.astype('float16') * 1/255
    xtest = xtest.astype('float16') * 1/255
    xval = xval.astype('float16') * 1/255

    trainSetSize = len(xtrain)
    testSetSize = len(xtest)
    valSetSize = len(xval)

    xtrain = xtrain.reshape((trainSetSize, 32, 32, 3))
    xtest = xtest.reshape((testSetSize, 32, 32, 3))
    xval = xval.reshape((valSetSize,32, 32, 3))

    ytrain = utils.to_categorical(ytrain)
    ytest = utils.to_categorical(ytest)
    yval = utils.to_categorical(yval)

    print('After preprocessing.. normalize to 1, OHE, reshaping')
    print( 'Shape of CIFAR Training Set Inputs: ' + str(xtrain.shape) )
    print( 'Shape of CIFAR Training Set Labels: ' + str(ytrain.shape) )
    print( 'Shape of CIFAR Test Set Inputs: ' + str(xtest.shape) )


    mymodel.compile(optimizer=keras.optimizers.RMSprop(), loss='categorical_crossentropy', metrics=["accuracy"])

    mycallbacksfunction = create_callbacks(ExpName)

   # history = mymodel.fit(xtrain, ytrain, epochs=2, batch_size=128,
                         # validation_data=(xval,yval),
                         # verbose=1, 
                         # callbacks=mycallbacksfunction
                         # )

    mybatchsize = 128
    history = mymodel.fit_generator( datagen.flow(xtrain, ytrain, batch_size=mybatchsize),
                    steps_per_epoch=xtrain.shape[0] / mybatchsize, 
                    epochs=15,
                    verbose=1,
                    validation_data=(xval,yval)
                      )

    #save to disk
    model_json = mymodel.to_json()
    with open('cifar_1B_softmax_model' + str(ExpName) + 's.json', 'w') as json_file:
        json_file.write(model_json)
    mymodel.save_weights('cifar_1B_softmax_model'  + str(ExpName) + 's.h5') 
   # files.download('cifar_1B_softmax_model' + str(ExpName) + 's.json')
   # files.download('cifar_1B_softmax_model'  + str(ExpName) + 's.h5')
    

    myTESTloss, myTESTaccuracy = print_model_metrics( mymodel, xtest, ytest )
    myVALloss, myVALaccuracy = print_model_metrics( mymodel, xval, yval )
    myTRAINloss, myTRAINaccuracy = print_model_metrics( mymodel, xtrain, ytrain)

    myTESTaccuracy = convertprecision(myTESTaccuracy)
    myVALaccuracy = convertprecision(myVALaccuracy)
    myTRAINaccuracy = convertprecision(myTRAINaccuracy)
    

    print(' \n RESULTS')
    print('TRAINING accuracy = ' + str(myTRAINaccuracy ))
    print('VAL accuracy = ' + str(myVALaccuracy))
    print('TEST accuracy = ' + str(myTESTaccuracy))
    
    

    return myTRAINaccuracy, myVALaccuracy, myTESTaccuracy


In [0]:
#Find insights about a network such as how deep is the network, memorizationcapacity, generalizationcapacity, noofCNNs, kernalsize, amountofnonlinearity
def extractNetworkInsights(anetwork, verbose=0):
    #print('Analzing the Network insights')
    layercount = 0
    layercountCNN = 0
    for alayer in anetwork.layers:
      layercount = layercount + 1
      aconfig = alayer.get_config()
      if (verbose==0):
        print(str(aconfig) )
      if (verbose==1):  
        if 'CNN' in str(aconfig.get("name")):
          layercountCNN = layercountCNN + 1 
          print( str(layercount) + '   ' + str(aconfig.get("name") +  '   Filters=' + str(aconfig.get("filters") )))
        elif ('Pool' in str(aconfig.get("name"))):
          print( str(layercount) + '   ' + str(aconfig.get("name") +  '   PoolSize=' + str(aconfig.get("pool_size") )))
        elif 'dropout' in aconfig.get("name"):
          print( str(layercount) + '   ' + str(aconfig.get("name") +  '   Rate=' + str(aconfig.get("rate") )))
        elif 'Dense' in aconfig.get("name"):
          print( str(layercount) + '   ' + str(aconfig.get("name") +  '   Units=' + str(aconfig.get("units") )) )         
        else:
          print( str(layercount) + '   ' + str(aconfig.get("name")))
    return str(layercountCNN) + ' CNN layers'

In [0]:
ExpName = 9020
islistcreated = False
ListofNetworks = list()
nooffiltersstarting =  32
networkwidthTamperRate = 2
noofDenselayers = 2
noofDensenodes = 64 
stride = 1
maxpool = 2
noofPARALLELbranches = 1
noofCNNlayers = 2
kernalsize = 3

In [0]:

#def createANetwork(ExpNo, noofCNNlayers, noofDenselayers, noofCNNFilters, noofDensenodes  ,                   networkwidthTamperRate, stride, maxpool, noofPARALLELbranches , kernalsize, dropoutrateinCNN=0, dropoutrateinFC=0.2):

for iterate in range( 0, 1):
  
    ExpName = ExpName + 1
    generatedNetwork =  createANetwork(ExpName, noofCNNlayers,noofDenselayers, nooffiltersstarting, noofDensenodes  ,networkwidthTamperRate, stride, maxpool,  noofPARALLELbranches, kernalsize, dropoutrateinCNN=0, dropoutrateinFC=0.2 )
    aTRAINaccuracy, aVALaccuracy, aTESTaccuracy = runaexpeirment(ExpName, generatedNetwork )
    atuple = tuple((ExpName, aTRAINaccuracy, aVALaccuracy, aTESTaccuracy ,  noofCNNlayers,noofDenselayers,  nooffiltersstarting, noofDensenodes  ,networkwidthTamperRate  , stride, maxpool, noofPARALLELbranches , kernalsize, generatedNetwork))

    ListofNetworks.append(atuple)

    print(" ADDED")
    print(atuple)

In [0]:

print('-----------------------------------------------------------')
print('Model No,         Train Acc,       Val Acc,       Test Acc')
for sometuple in ListofNetworks:
    print(str(sometuple[0]) +'             ' , str(sometuple[1]) + '            ' , str(sometuple[2])+ '         ', str(sometuple[3]) +',  ' + str( sometuple[13]) )

In [0]:
extractNetworkInsights(ListofNetworks[0][13], verbose=1)

In [0]:
files.download('cifar_1B_softmax_model' + str(ExpName) + 's.json')


In [0]:
files.download('cifar_1B_softmax_model'  + str(ExpName) + 's.h5')

In [0]:
(xtraincheck, ytraincheck), (xtestcheck, ytestcheck) = datasets.cifar10.load_data()
myimageJ = xtraincheck[114]
myimageJnormalized = myimageJ.astype('float16') * 1/255

In [0]:
myloadedmodel =  createANetwork(ExpName, noofCNNlayers,noofDenselayers, nooffiltersstarting, noofDensenodes  ,networkwidthTamperRate, stride, maxpool,  noofPARALLELbranches, kernalsize )

print(myloadedmodel.predict(xtraincheck[400:401]))
